In [1]:
import numpy as np
import math
import pandas as pd
import tqdm
from scipy.stats import chi2, t
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, exp
from biogeme import results as res
from IPython.display import display, Markdown, Latex
from biogeme.models import loglogit
from biogeme import biogeme
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from biogeme.models import piecewise_formula
from sklearn.model_selection import train_test_split

In [2]:
def likelihood_ratio_test(model_initial, model_next):
    LL_logit = model_initial.data.logLike
    LL_nested_existing = model_next.data.logLike
    LR = -2 * (LL_logit - LL_nested_existing)
    print(f'Statistic for the LR test: {LR:.3g}')

    dof = model_next.data.nparam - model_initial.data.nparam
    print(f'Degrees of freedom: {dof}')

    print(f'Threshold for the test: {chi2.isf(0.05, dof):.3g}')

    lr_test_existing = model_next.likelihood_ratio_test(model_initial, significance_level=0.05)
    print(f'{lr_test_existing.statistic=:.3g}')
    print(f'{lr_test_existing.threshold=:.3g}')
    print(lr_test_existing.message)

In [3]:
file_path = '../data_CSV/final_array.csv'
df = pd.read_csv(file_path)

/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/482401865.py:2: DtypeWarning: Columns (8,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
df['unit_price']= df['price_driver']/df['distance_BBC']
df['airport_distance'] = df['airport_distance']/1000
df['trainstation_distance'] = df['trainstation_distance']/1000
df['electric_distance'] = df['electric_distance']/1000
df['bicycle_parking_distance'] = df['bicycle_parking_distance']/1000
df['tram_stop_distance'] = df['tram_stop_distance']/1000
df['subway_entrance_distance'] = df['subway_entrance_distance']/1000
df['bus_stop_distance'] = df['bus_stop_distance']/1000
df['bicycle_rental_distance'] = df['bicycle_rental_distance']/1000
df['fuel_distance'] = df['fuel_distance']/1000
df['dens_pop'] = df['dens_pop']/100  # de habitants par km2 a habitants par hectare 

print(df['bus_stop_distance'])

0         0.084061
1         0.163721
2         0.007305
3         0.038377
4         0.054396
            ...   
169278    0.033107
169279    0.131402
169280    0.084061
169281    0.029789
169282    0.079733
Name: bus_stop_distance, Length: 169283, dtype: float64


In [5]:
def time_to_minutes1(time_str):
    time_obj = datetime.strptime(time_str, '%H:%M:%S')  # Parse the string to a datetime object
    return time_obj.hour * 60 + time_obj.minute  # Convert to minutes

def time_to_minutes2(time_obj):
    return time_obj.hour * 60 + time_obj.minute  # Convert to minutes

In [6]:

# Filter out rows where `seats_booked` is not valid (negative values)
df = df[df['seats_booked'] >= 0]

# Ensure seats_booked is an integer
df['seats_booked'] = df['seats_booked'].astype(int)

# Create a list to store rows for the new DataFrame
result_rows = []

for _, row in df.iterrows():
    seats = row['seats_booked']
    row_data = row.to_dict()  # Convert the row to a dictionary
    if seats == 0:
        # Add the row with binary_seats = 0
        row_data['binary_seats'] = 0
        result_rows.append(row_data)
    elif seats >= 1:
        # Add the row with binary_seats = 1
        row_data['binary_seats'] = 1
        result_rows.append(row_data)

# Convert the list of rows to a DataFrame
result = pd.DataFrame(result_rows)


In [7]:
# Convert time strings to seconds since midnight
# Convert time strings to time objects
result['departure_time'] = result['departure_time'].apply(lambda x: x.split()[1])
result['departure_time'] = result['departure_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())

# Define custom function for time categorization
def categorize_time(time_value):
    if time_value < datetime.strptime("04:00:00", "%H:%M:%S").time():
        return "Night"
    elif time_value < datetime.strptime("07:30:00", "%H:%M:%S").time():
        return "Morning_Normal"
    elif time_value < datetime.strptime("09:30:00", "%H:%M:%S").time():
        return "Morning_Pick"
    elif time_value < datetime.strptime("16:30:00", "%H:%M:%S").time():
        return "Mid_day"
    elif time_value < datetime.strptime("19:30:00", "%H:%M:%S").time():
        return "Evening_Pick"
    else:
        return "Evening_Normal"

# Apply the function to create the time_category column
result['time_category'] = result['departure_time'].apply(categorize_time)

# Perform one-hot encoding (binary columns for each category)
result = pd.get_dummies(result, columns=['time_category'], prefix='time')

# Ensure that the columns are binary (1 or 0)
result = result.astype({"time_Night": "int", "time_Morning_Normal": "int", "time_Morning_Pick": "int", "time_Mid_day": "int", "time_Evening_Pick": "int", "time_Evening_Normal": "int"})

In [8]:
# Define custom function for time categorization
def categorize_denspop(denspop_value):
    if denspop_value < 80:
        return "dens_a"
    elif denspop_value < 160:
        return "dens_b"
    elif denspop_value < 240:
        return "dens_c"
    elif denspop_value < 320:
        return "dens_d"
    else:
        return "dens_e"

# Apply the function to create the time_category column
result['denspop_category'] = result['dens_pop'].apply(categorize_denspop)

# Perform one-hot encoding (binary columns for each category)
result = pd.get_dummies(result, columns=['denspop_category'], prefix='pop')

# Ensure that the columns are binary (1 or 0)
result = result.astype({"pop_dens_a": "int", "pop_dens_b": "int","pop_dens_c": "int","pop_dens_d": "int","pop_dens_e": "int"})
print("pop_dens_a:", result['pop_dens_a'].sum())
print("pop_dens_b:", result['pop_dens_b'].sum())
print("pop_dens_c:", result['pop_dens_c'].sum())
print("pop_dens_d:", result['pop_dens_d'].sum())
print("pop_dens_e:", result['pop_dens_e'].sum())

pop_dens_a: 57375
pop_dens_b: 35490
pop_dens_c: 22522
pop_dens_d: 53213
pop_dens_e: 682


In [9]:
result = result.drop(columns=['arrival'])
result = result.drop(columns=['departure'])
result = result.drop(columns=['route'])
result = result.drop(columns=['departure_date_num'])
result = result.drop(columns=['announce_published_date'])
result = result.drop(columns=['departure_time'])
result = result.drop(columns=['car_mark'])
result = result.drop(columns=['car_model'])
result = result.drop(columns=['departure_date'])
result = result.drop(columns=['trip_description'])
result = result.drop(columns=['announce_views'])
result = result.drop(columns=['comment_average_note'])
result = result.drop(columns=['geometry'])
result = result.drop(columns=['driver_age'])
result = result.drop(columns=['driving_quality'])
result = result.drop(columns=['experience'])
result = result.drop(columns=['member_since_month'])
result = result.drop(columns=['member_since_year'])
result = result.drop(columns=['published_ads'])
result = result.drop(columns=['departure_strike'])
result = result.drop(columns=['commission'])
df_cleaned = result.dropna()
database = db.Database("lpmc09", df_cleaned)

In [10]:
#Declaration of the variables
#DensityPop = Variable('dens_pop')
Departure = Variable('Minutes')
Safety = Variable('norm_weighted_sum')
FuelStation1 = Variable('fuel_distance')
FuelStation2 = Variable('fuel_density')
BikeRent1 = Variable('bicycle_rental_distance')
BikeRent2 = Variable('bicycle_rental_density')
BusStop1 = Variable('bus_stop_distance')
BusStop2 = Variable('bus_stop_density')
SubwayEntr1 = Variable('subway_entrance_distance')
SubwayEntr2 = Variable('subway_entrance_density')
TramStop1 = Variable('tram_stop_distance')
TramStop2 = Variable('tram_stop_density')
BikePark1 = Variable('bicycle_parking_distance')
BikePark2 = Variable('bicycle_parking_density')
ElecPark1 = Variable('electric_distance')
ElecPark2 = Variable('electric_density')
TrainStation1 = Variable('trainstation_distance')
TrainStation2 = Variable('trainstation_density')
Airport1 = Variable('airport_distance')
Airport2 = Variable('airport_density')
CarpoolPark1 = Variable('parkCarpooling_distance')
CarpoolPark2 = Variable('parkCarpooling_density')
PplusR1 = Variable('parkRelais_distance')
PplusR2 = Variable('parkRelais_density')
Distance = Variable('distance_BBC')
departure_group_2 = Variable('time_Morning_Normal')
departure_group_3 = Variable('time_Morning_Pick')
departure_group_4 = Variable('time_Mid_day')
departure_group_5 = Variable('time_Evening_Pick')
departure_group_6 = Variable('time_Evening_Normal')
DensPop = Variable('dens_pop')
DensPop_group_2 = Variable('pop_dens_b')
DensPop_group_3 = Variable('pop_dens_c')
DensPop_group_4 = Variable('pop_dens_d')
DensPop_group_5 = Variable('pop_dens_e')
Centre = Variable('Centre')
Banlieue = Variable('Banlieue')
Price = Variable('unit_price')

#thresholds = [None, 500, 1500, 5000, 10000, 20000, 30000, None]
thresholds = [None, 5000, None]
formulaDensPop = piecewise_formula(variable=DensPop, thresholds=thresholds)
thresholds=[None,450,570,None]#thresholds = [
  ##  None,
  #  time_to_minutes2(datetime.strptime("04:00:00", "%H:%M:%S").time()),
  #  time_to_minutes2(datetime.strptime("07:30:00", "%H:%M:%S").time()),
  #  time_to_minutes2(datetime.strptime("09:30:00", "%H:%M:%S").time()),
  #  time_to_minutes2(datetime.strptime("16:30:00", "%H:%M:%S").time()),
  #  time_to_minutes2(datetime.strptime("19:30:00", "%H:%M:%S").time()),
   # None
#]
formulaTime = piecewise_formula(variable=Departure, thresholds=thresholds)

In [11]:
ChosenAlternative = Variable('binary_seats') #try with the duplicate version and try without

In [12]:
#Definition of the parameters

#beta_densitypop = Beta('beta_densitypop',0,None,None,0)
beta_safety = Beta('beta_safety',0,None,None,0)
beta_fueldist = Beta('beta_fueldist',0,None,None,0)
beta_fueldens = Beta('beta_fueldens',0,None,None,0)
beta_bikerentdist = Beta('beta_bikerentdist',0,None,None,0)
beta_bikerentdens = Beta('beta_bikerentdens',0,None,None,0)
beta_busstopdist = Beta('beta_busstopdist',0,None,None,0)
beta_busstopdens = Beta('beta_busstopdens',0,None,None,0)
beta_subentrdist = Beta('beta_subentrdist',0,None,None,0)
beta_subentrdens = Beta('beta_subentrdens',0,None,None,0)
beta_tramstopdist = Beta('beta_tramstopdist',0,None,None,0)
beta_tramstopdens = Beta('beta_tramstopdens',0,None,None,0)
beta_bikeparkdist = Beta('beta_bikeparkdist',0,None,None,0)
beta_bikeparkdens = Beta('beta_bikeparkdens',0,None,None,0)
beta_elecparkdist = Beta('beta_elecparkdist',0,None,None,0)
beta_elecparkdens = Beta('beta_elecparkdens',0,None,None,0)
beta_traindist = Beta('beta_traindist',0,None,None,0)
beta_traindens = Beta('beta_traindens',0,None,None,0)
beta_airportdist = Beta('beta_airportdist',0,None,None,0)
beta_airportdens = Beta('beta_airportdens',0,None,None,0)
beta_carpoolparkdist = Beta('beta_carpoolparkdist',0,None,None,0)
beta_carpoolparkdens = Beta('beta_carpoolparkdens',0,None,None,0)
beta_parkRelaisdist = Beta('beta_parkRelaisdist',0,None,None,0)
beta_parkRelaisdens = Beta('beta_parkRelaisdens',0,None,None,0)
beta_distance = Beta('beta_distance',0,None,None,0) 
beta_departure_2 = Beta('beta_departure_2', 0, None, None, 0) #piecewise (voir mmb)
beta_departure_3 = Beta('beta_departure_3', 0, None, None, 0)
beta_departure_4 = Beta('beta_departure_4', 0, None, None, 0)
beta_departure_5 = Beta('beta_departure_5', 0, None, None, 0)
beta_departure_6 = Beta('beta_departure_6', 0, None, None, 0)
beta_densPop_2 = Beta('beta_densPop_2', 0, None, None, 0)
beta_densPop_3 = Beta('beta_densPop_3', 0, None, None, 0)
beta_densPop_4 = Beta('beta_densPop_4', 0, None, None, 0)
beta_densPop_5 = Beta('beta_densPop_5', 0, None, None, 0)
beta_price = Beta('beta_price',0,None,None,0)
beta_centre= Beta('beta_centre',0,None,None,0)
beta_banlieue = Beta('beta_banlieue',0,None,None,0)
ASC_1 = Beta('ASC_1', 0, None, None, 0)

In [13]:
N = df_cleaned.shape[0]
J = 2

init_log_likelihood = N*math.log(1/J)
print(f"Initial Log-Likelihood: {init_log_likelihood}")

Initial Log-Likelihood: -117337.34101954866


In [14]:
from scipy.stats import chi2

def likelihood_ratio_test(model_1, model_0):
    """
    Effectue un test du rapport de vraisemblance (LRT) entre deux modèles.
    
    Args:
    model_1 (bio.BIOGEME): Le modèle complet.
    model_0 (bio.BIOGEME): Le modèle nul (ou modèle restreint).
    
    Returns:
    dict: Résultats du test LRT, y compris la statistique du test et la p-value.
    """
    # Estimation du modèle complet
    result_1 = model_1.estimate()
    log_likelihood_1 = result_1.data.logLike
    
    # Estimation du modèle nul
    result_0 = model_0.estimate()
    log_likelihood_0 = result_0.data.logLike
    
    # Calcul de la statistique du test LRT
    lrt_statistic = -2 * (log_likelihood_0 - log_likelihood_1)
    
    # Degrés de liberté (différence du nombre de paramètres entre les modèles)
    df = result_1.getEstimatedParameters().shape[0] - result_0.getEstimatedParameters().shape[0]
    
    # Calcul de la p-value
    p_value = chi2.sf(lrt_statistic, df)
    
    return {
        'lrt_statistic': lrt_statistic,
        'degrees_of_freedom': df,
        'p_value': p_value
    }



In [15]:
# Step 1: Assign a unique GROUP ID to each (departure, arrival, departure_date_num)
#df_cleaned['GROUP'] = df_cleaned.groupby(['departure', 'arrival', 'departure_date_num']).ngroup()  # Ensure IDs start from 1

# Step 2: Count how many alternatives each GROUP has
#group_sizes = df_cleaned.groupby('GROUP').size().to_dict()  # {GROUP_ID: number of alternatives}

# Étape 3: Définir la taille max
#max_alternatives = max(group_sizes.values()) 

# Étape 4: Générer les colonnes avail_1 ... avail_max_alternatives
#avail_columns = [f"avail_{i}" for i in range(max_alternatives)]
#availability_matrix = np.zeros((len(df), max_alternatives), dtype=int)  # Matrice remplie de 0
 
# Étape 5: Remplir les colonnes
#for idx, row in df.iterrows():
    #group_id = row['GROUP']
    #size = group_sizes[group_id]  # Nombre d'alternatives pour ce groupe
    #availability_matrix[idx, :size] = 1  # Met 1 dans les premières colonnes correspondant à la taille du groupe

# Étape 6: Ajouter au dataframe
#df_availability = pd.DataFrame(availability_matrix, columns=avail_columns)
#df_cleaned = pd.concat([df_cleaned, df_availability], axis=1)

# Afficher le résultat
#print(df_cleaned['avail_100'])


In [16]:
# Séparer les données en deux ensembles (80 % - 20 %)
train_data, test_data = train_test_split(df_cleaned, test_size=0.2, random_state=None)
# Define the Biogeme database for the training data
database_train = db.Database('train', train_data)

print(len(train_data))

135425


In [17]:
V_seats_00 = (
    ASC_1 + beta_price*Price +
    beta_bikerentdist*BikeRent1 +
    beta_bikeparkdens*BikePark2 +
    beta_tramstopdist*TramStop1 +
    beta_busstopdist*BusStop1+
    beta_busstopdens*BusStop2+
    beta_elecparkdens*ElecPark2+
    beta_centre*Centre +
    beta_banlieue*Banlieue +
    beta_densPop_2*DensPop_group_2 +
    beta_densPop_3*DensPop_group_3 +
    beta_densPop_4*DensPop_group_4 +
    beta_densPop_5*DensPop_group_5 +
    beta_departure_2*departure_group_2 +
    beta_departure_3*departure_group_3 +
    beta_departure_4*departure_group_4 +
    beta_departure_5*departure_group_5 +
    beta_departure_6*departure_group_6
)

# Model specification
V = {1: V_seats_00, 0: 0}  # Utility functions: 1 for seats, 0 is the baseline
av = {1: 1, 0: 1}  # Availability: both alternatives are always available

# Log-likelihood function
logprob = loglogit(V, None, ChosenAlternative)

# Create the Biogeme object
model_00 = bio.BIOGEME(database_train, logprob)

# Model name
model_00.modelName = 'binary_logit_seats'

results = {}

results['binary_logit_seats'] = model_00.estimate()
model_estimation_00 = results['binary_logit_seats']
print(results['binary_logit_seats'].printGeneralStatistics())
#likelihood_ratio_test(model_estimation_01 ,model_estimation_6 )
results['binary_logit_seats'].getEstimatedParameters()

Number of estimated parameters:	19
Sample size:	135425
Excluded observations:	0
Init log likelihood:	-92240.67
Final log likelihood:	-92232.98
Likelihood ratio test for the init. model:	15.37135
Rho-square for the init. model:	8.33e-05
Rho-square-bar for the init. model:	-0.000123
Akaike Information Criterion:	184504
Bayesian Information Criterion:	184690.5
Final gradient norm:	5.7914E-01
Nbr of threads:	4



/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/3567805743.py:39: DeprecationWarning: printGeneralStatistics is deprecated; use print_general_statistics instead.
  print(results['binary_logit_seats'].printGeneralStatistics())
/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/3567805743.py:41: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results['binary_logit_seats'].getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_1,0.113264,0.113719,0.995998,3.192509e-01
beta_banlieue,-0.168464,0.079486,-2.119413,3.405560e-02
beta_bikeparkdens,-0.000185,0.000100,-1.851716,6.406664e-02
beta_bikerentdist,-0.011775,0.002614,-4.504639,6.648594e-06
beta_busstopdens,0.004004,0.000523,7.655837,1.909584e-14
beta_busstopdist,-0.464747,0.082278,-5.648505,1.618492e-08
beta_centre,-0.000611,0.081322,-0.007518,9.940014e-01
beta_densPop_2,0.135709,0.019664,6.901414,5.148770e-12
beta_densPop_3,0.023973,0.024128,0.993586,3.204246e-01
beta_densPop_4,0.159839,0.024410,6.548052,5.829248e-11


In [18]:
#V_seats_01 = (
  #  ASC_1 + beta_price*Price +
  #  beta_bikerentdist*BikeRent1 +
   # beta_tramstopdist*TramStop1 +
   # beta_busstopdist*BusStop1+
   # beta_busstopdens*BusStop2+
  #  beta_banlieue*Banlieue +
  #  beta_densPop_2*DensPop_group_2 +
  #  beta_densPop_3*DensPop_group_3 +
  #  beta_densPop_4*DensPop_group_4 +
  #  beta_densPop_5*DensPop_group_5 +
  #  beta_departure_2*departure_group_2 +
  #  beta_departure_3*departure_group_3 +
   # beta_departure_4*departure_group_4 +
   # beta_departure_5*departure_group_5 +
   # beta_departure_6*departure_group_6
#)

# Model specification
#V = {1: V_seats_01, 0: 0}  # Utility functions: 1 for seats, 0 is the baseline
#av = {1: 1, 0: 1}  # Availability: both alternatives are always available

# Log-likelihood function
#logprob = loglogit(V, None, ChosenAlternative)

# Create the Biogeme object
#model_01 = bio.BIOGEME(database_train, logprob)

# Model name
#model_01.modelName = 'binary_logit_seats'

#results = {}

#results['binary_logit_seats'] = model_01.estimate()
#model_estimation_01 = results['binary_logit_seats']
#print(results['binary_logit_seats'].printGeneralStatistics())
#likelihood_ratio_test(model_estimation_01 ,model_estimation_6 )
#results['binary_logit_seats'].getEstimatedParameters()

In [19]:
N = train_data.shape[0]
J = 2

init_log_likelihood = N*math.log(1/J)
print(f"Initial Log-Likelihood: {init_log_likelihood}")

Initial Log-Likelihood: -93869.4569273306


In [20]:
V_seats_01 = (
    ASC_1 + beta_price*Price +
    beta_bikerentdist*BikeRent1 +
    beta_tramstopdist*TramStop1 +
    beta_busstopdist*BusStop1+
    beta_busstopdens*BusStop2+
    beta_banlieue*Banlieue +
    beta_densPop_2*DensPop_group_2 +
    beta_densPop_3*DensPop_group_3 +
    beta_densPop_4*DensPop_group_4 +
    beta_densPop_5*DensPop_group_5 +
    beta_departure_2*departure_group_2 +
    beta_departure_3*departure_group_3 +
    beta_departure_4*departure_group_4 +
    beta_departure_5*departure_group_5 +
    beta_departure_6*departure_group_6
)

# Model specification
V = {1: V_seats_01, 0: 0}  # Utility functions: 1 for seats, 0 is the baseline
av = {1: 1, 0: 1}  # Availability: both alternatives are always available

# Log-likelihood function
logprob = loglogit(V, None, ChosenAlternative)

# Create the Biogeme object
model_01 = bio.BIOGEME(database_train, logprob)

# Model name
model_01.modelName = 'binary_logit_seats'

results = {}

results['binary_logit_seats'] = model_01.estimate()
model_estimation_01 = results['binary_logit_seats']
print(results['binary_logit_seats'].printGeneralStatistics())
#likelihood_ratio_test(model_estimation_01 ,model_estimation_6 )
results['binary_logit_seats'].getEstimatedParameters()

Number of estimated parameters:	16
Sample size:	135425
Excluded observations:	0
Init log likelihood:	-92280.52
Final log likelihood:	-92236.44
Likelihood ratio test for the init. model:	88.15916
Rho-square for the init. model:	0.000478
Rho-square-bar for the init. model:	0.000304
Akaike Information Criterion:	184504.9
Bayesian Information Criterion:	184661.9
Final gradient norm:	1.0389E-01
Nbr of threads:	4



/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/2064514595.py:36: DeprecationWarning: printGeneralStatistics is deprecated; use print_general_statistics instead.
  print(results['binary_logit_seats'].printGeneralStatistics())
/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/2064514595.py:38: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results['binary_logit_seats'].getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_1,0.101489,0.081527,1.244840,2.131906e-01
beta_banlieue,-0.137545,0.018806,-7.314105,2.591261e-13
beta_bikerentdist,-0.010435,0.002522,-4.136720,3.523051e-05
beta_busstopdens,0.003168,0.000403,7.856529,3.996803e-15
beta_busstopdist,-0.477924,0.082334,-5.804715,6.447561e-09
beta_densPop_2,0.142821,0.019413,7.356964,1.880718e-13
beta_densPop_3,0.007079,0.023117,0.306211,7.594442e-01
beta_densPop_4,0.163598,0.024310,6.729623,1.701017e-11
beta_densPop_5,0.162014,0.089726,1.805640,7.097457e-02
beta_departure_2,0.209220,0.052689,3.970865,7.161226e-05


In [21]:
# Modèle nul (par exemple, un modèle sans variables explicatives)
V_null = {1: ASC_1, 0: 0}  # Utilité sans aucune variable explicative
logprob_null = loglogit(V_null, None, ChosenAlternative)

# Création du modèle nul
model_null = bio.BIOGEME(database_train, logprob_null)

# Application du test LRT
lrt_results = likelihood_ratio_test(model_01, model_null)

print("LRT Statistique:", lrt_results['lrt_statistic'])
print("Degrés de liberté:", lrt_results['degrees_of_freedom'])
print("P-value:", lrt_results['p_value'])

You have not defined a name for the model. The output .py are named from the model name. The default is [biogemeModelDefaultName]


LRT Statistique: 2736.06176424204
Degrés de liberté: 15
P-value: 0.0


/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/2058997342.py:26: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  df = result_1.getEstimatedParameters().shape[0] - result_0.getEstimatedParameters().shape[0]
/var/folders/ym/s4xnst_s0y769j7rl84pgpww0000gn/T/ipykernel_24780/2058997342.py:26: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  df = result_1.getEstimatedParameters().shape[0] - result_0.getEstimatedParameters().shape[0]


In [29]:
# Liste des variables utilisées dans le modèle
model_variables = [
    'unit_price','bicycle_rental_distance','tram_stop_distance','bus_stop_distance','bus_stop_density',
    'Banlieue','time_Night','time_Morning_Normal','time_Morning_Pick','time_Mid_day',
    'time_Evening_Pick','time_Evening_Normal','pop_dens_a','pop_dens_b','pop_dens_c','pop_dens_d','pop_dens_e'
]

# Extraction des variables spécifiques
data_model = database_train.data[model_variables]

# Calcul de la matrice de corrélation
correlation_matrix = data_model.corr()

# Affichage du tableau des corrélations
print(correlation_matrix)


                         unit_price  bicycle_rental_distance  \
unit_price                 1.000000                -0.046298   
bicycle_rental_distance   -0.046298                 1.000000   
tram_stop_distance        -0.037503                 0.609670   
bus_stop_distance          0.010528                 0.205269   
bus_stop_density           0.033933                -0.444245   
Banlieue                  -0.044589                 0.293967   
time_Night                 0.024588                -0.006802   
time_Morning_Normal        0.004951                 0.026278   
time_Morning_Pick         -0.018835                 0.038472   
time_Mid_day              -0.046974                 0.011379   
time_Evening_Pick          0.018944                -0.021445   
time_Evening_Normal        0.057500                -0.050970   
pop_dens_a                -0.062302                 0.553966   
pop_dens_b                 0.035844                -0.186197   
pop_dens_c                 0.011844     

In [23]:
# Récupération des paramètres estimés
beta_estimates = results['binary_logit_seats'].get_beta_values()

V_seats_01_test = (
    beta_estimates['ASC_1'] + beta_estimates['beta_price'] * test_data['unit_price'] +
    beta_estimates['beta_bikerentdist'] * test_data['bicycle_rental_distance'] +
    beta_estimates['beta_tramstopdist'] * test_data['tram_stop_distance'] +
    beta_estimates['beta_busstopdist'] * test_data['bus_stop_distance'] +
    beta_estimates['beta_busstopdens'] * test_data['bus_stop_density'] +
    beta_estimates['beta_banlieue'] * test_data['Banlieue'] +
    beta_estimates['beta_densPop_2'] * test_data['pop_dens_b'] +
    beta_estimates['beta_densPop_3'] * test_data['pop_dens_c'] +
    beta_estimates['beta_densPop_4'] * test_data['pop_dens_d'] +
    beta_estimates['beta_densPop_5'] * test_data['pop_dens_e'] +
    beta_estimates['beta_departure_2'] * test_data['time_Morning_Normal'] +
    beta_estimates['beta_departure_3'] * test_data['time_Morning_Pick'] +
    beta_estimates['beta_departure_4'] * test_data['time_Mid_day'] +
    beta_estimates['beta_departure_5'] * test_data['time_Evening_Pick'] +
    beta_estimates['beta_departure_6'] * test_data['time_Evening_Normal']

)

# Utilité pour l'alternative 0 (baseline) est toujours 0
V_0_test = np.zeros_like(V_seats_01_test)

# Calcul des probabilités pour chaque observation dans les données de test
probabilities_test = np.exp(V_seats_01_test) / (np.exp(V_seats_01_test) + np.exp(V_0_test))

# Affichage des probabilités simulées
print(probabilities_test)

# Calcul des prédictions : Si la probabilité pour l'alternative 1 est plus grande que 0.5, on prédit l'alternative 1, sinon on prédit l'alternative 0
predictions = (probabilities_test > 0.5).astype(int)

# Comparer avec l'alternative réellement choisie (assumant qu'elle soit dans la colonne 'ChosenAlternative')
correct_predictions = (predictions == test_data['binary_seats']).sum()

# Calcul du pourcentage de prédictions correctes
accuracy = (correct_predictions / len(test_data)) * 100

# Affichage du pourcentage de prédictions correctes
print(f"Pourcentage de prédictions correctes : {accuracy:.2f}%")


43854     0.467541
14391     0.459454
16325     0.444639
134169    0.478425
40767     0.529821
            ...   
54466     0.377317
73485     0.461528
57071     0.255519
39188     0.578021
25106     0.497976
Length: 33857, dtype: float64
Pourcentage de prédictions correctes : 56.13%


In [24]:
r = {
    'seats_booked':len(train_data[train_data['binary_seats']==1]),
    'no_seats_booked':len(train_data[train_data['binary_seats']==0])
}
Sg= r.values()
S_train = len(train_data)

# Calculate the market share for seats booked and not booked
market_share_booked = r['seats_booked'] / S_train * 100
market_share_no_booked = r['no_seats_booked'] / S_train* 100

# Print the market shares
print(f"Market share of seats booked: {market_share_booked:.2f}%")
print(f"Market share of no seats booked: {market_share_no_booked:.2f}%")

# Initialize a variable to store the sum of logs
log_sum = 0
count1 =0
count2 =0
# Loop through each prediction and add the corresponding log value
for prediction, actual in zip(predictions, test_data['binary_seats']):
    if prediction == 1:  # If predicted "seats booked"
        log_sum += np.log(market_share_booked/100)  # Add log of the market share for seats booked
        count1+=1
    else:  # If predicted "no seats booked"
        log_sum += np.log(market_share_no_booked/100)  # Add log of the market share for no seats booked
        count2+=1

N = test_data.shape[0]
J = 2

print(f"Initial Log-Likelihood: {init_log_likelihood}")
likelihood_comp =N*math.log(1/J)
# Print the final sum of logs
print(f"Sum of log market shares: {log_sum}")
print(f"Random log likelihood: {likelihood_comp}")
print(count1,count2)

Market share of seats booked: 46.87%
Market share of no seats booked: 53.13%
Initial Log-Likelihood: -93869.4569273306
Sum of log market shares: -23003.779461791844
Random log likelihood: -23467.884092218068
12693 21164


In [25]:
import scipy.stats as stats
def chi2_likelihood_ratio_test(log_likelihood_model, log_likelihood_random, df):
    """
    Effectue un test du rapport de vraisemblance (LRT) avec la statistique chi-deux.
    
    Parameters:
    - log_likelihood_model: log-vraisemblance de votre modèle.
    - log_likelihood_random: log-vraisemblance du modèle aléatoire.
    - df: différence du nombre de paramètres entre les deux modèles (k).
    
    Returns:
    - chi2_statistic: la statistique du test (rapport de vraisemblance).
    - p_value: la p-value du test
    """
    # Calcul de la statistique chi-deux du rapport de vraisemblance
    chi2_statistic = -2 * (log_likelihood_random - log_likelihood_model)
    
    # Comparer avec la distribution chi-deux avec `df` degrés de liberté
    p_value = 1 - stats.chi2.cdf(chi2_statistic, df)
    
    return chi2_statistic, p_value

df = 11  # Exemple, différence entre le nombre de paramètres (k)

chi2_statistic, p_value = chi2_likelihood_ratio_test(log_sum, likelihood_comp, df)

print(f"Statistique du test chi-deux: {chi2_statistic}")
print(f"P-value du test: {p_value}")



Statistique du test chi-deux: 928.2092608524472
P-value du test: 0.0


In [26]:
print(predictions)
print(test_data['binary_seats'])
print(test_data['binary_seats'].tolist())

43854     0
14391     0
16325     0
134169    0
40767     1
         ..
54466     0
73485     0
57071     0
39188     1
25106     0
Length: 33857, dtype: int64
43854     0
14391     1
16325     0
134169    0
40767     0
         ..
54466     0
73485     1
57071     1
39188     1
25106     1
Name: binary_seats, Length: 33857, dtype: int64
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,

In [27]:
from sklearn.metrics import confusion_matrix

def evaluate_predictions(y_true, y_pred):
    """
    Fonction pour évaluer la qualité des prédictions de réservation de sièges.
    
    Parameters:
    - y_true: Liste ou tableau des valeurs réelles (0 = pas réservé, 1 = réservé)
    - y_pred: Liste ou tableau des valeurs prédites (0 = pas réservé, 1 = réservé)
    
    Returns:
    - Aucune valeur retournée, mais affiche les résultats
    """
    # Calcul de la matrice de confusion
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    print(f"True Positives (TP): {tp} - Nombre de sièges réservés correctement prédits comme réservés.")
    print(f"True Negatives (TN): {tn} - Nombre de sièges non réservés correctement prédits comme non réservés.")
    print(f"False Positives (FP): {fp} - Nombre de sièges non réservés incorrectement prédits comme réservés.")
    print(f"False Negatives (FN): {fn} - Nombre de sièges réservés incorrectement prédits comme non réservés.")
    
    # Optionnel : Calcul de quelques métriques supplémentaires
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    print(f"\nMétriques supplémentaires :")
    print(f"Accuracy (Précision globale): {accuracy:.4f}")
    print(f"Precision (Précision sur les réservations): {precision:.4f}")
    print(f"Recall (Rappel sur les réservations): {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Exemple d'utilisation :
# y_true : les valeurs réelles (0 = pas réservé, 1 = réservé)
# y_pred : les valeurs prédites par votre modèle (0 = pas réservé, 1 = réservé)

# Exemple de données (y_true = réel, y_pred = prédit)
y_true = test_data['binary_seats']  # Par exemple : 1 = réservé, 0 = pas réservé
y_pred =  predictions# Prédictions du modèle

evaluate_predictions(y_true, y_pred)


True Positives (TP): 6860 - Nombre de sièges réservés correctement prédits comme réservés.
True Negatives (TN): 12143 - Nombre de sièges non réservés correctement prédits comme non réservés.
False Positives (FP): 5833 - Nombre de sièges non réservés incorrectement prédits comme réservés.
False Negatives (FN): 9021 - Nombre de sièges réservés incorrectement prédits comme non réservés.

Métriques supplémentaires :
Accuracy (Précision globale): 0.5613
Precision (Précision sur les réservations): 0.5405
Recall (Rappel sur les réservations): 0.4320
F1 Score: 0.4802


In [28]:
# Définition des paramètres à estimer
ASC = {i: Beta(f"ASC_{i}", 0, None, None, 1 if i == 1 else 0) for i in range(0, max_alternatives)}

# Définition des utilités pour chaque alternative
V = {i: ASC[i] + beta_price*Price + \
                beta_busstopdist*BusStop1 + \
                beta_busstopdens*BusStop2 + \
                beta_tramstopdist*TramStop1 + \
                beta_tramstopdens*TramStop2 + \
                beta_subentrdens*SubwayEntr2 + \
                beta_traindens*TrainStation2 + \
                beta_fueldens*FuelStation2 + \
                beta_safety*Safety + \
                beta_densPop_4*DensPop_group_4 + \
                beta_densPop_5*DensPop_group_5 + \
                beta_densPop_6*DensPop_group_6 + \
                beta_departure_2 * departure_group_2 + \
                beta_departure_3 * departure_group_3 + \
                beta_departure_4 * departure_group_4 + \
                beta_departure_5 * departure_group_5 + \
                beta_departure_6 * departure_group_6  for i in range(0, max_alternatives)}

availability = {i: Variable(f"avail_{i}") for i in range(0, max_alternatives)}

# Définition du modèle logit
logprob = loglogit(V, availability, ChosenAlternative)

# Création de l’objet Biogeme
biogeme = bio.BIOGEME(database_train, logprob)
biogeme.modelName = "MNL_Many_Groups"

# Estimation
results_train = biogeme.estimate()

print(results_train.printGeneralStatistics())
# Affichage des résultats
print(results_train.getEstimatedParameters())

# Get estimated parameters from the training model
beta_estimates = results_train.get_beta_values()

# Evaluate on the test set
database_test = db.Database('test', test_data)

# Create a Biogeme object for the test data
model_test = bio.BIOGEME(database_test, logprob)

# Simulate probabilities for the test set
simulated_probs = model_test.simulate(beta_estimates)

print(simulated_probs)

NameError: name 'max_alternatives' is not defined

In [ ]:
# Get estimated parameter values
betas = results_train.getBetaValues()

# Compute probabilities manually
test_data['V2'] = betas['ASC_1']+ betas['beta_price'] * test_data['unit_price'] + betas['beta_departure_2'] * test_data['time_Morning_Normal']+betas['beta_departure_3'] * test_data['time_Morning_Pick']+betas['beta_departure_4'] * test_data['time_Mid_day']+betas['beta_departure_5'] * test_data['time_Evening_Pick']+betas['beta_departure_6'] * test_data['time_Evening_Normal']
test_data['P1'] = 1 / (1 + np.exp(test_data['V2'])) # P1 = 1 / (1 + exp(V2))
test_data['P0'] = 1 - test_data['P1']  # Since P1 + P2 = 1

# Show probabilities
print(test_data[['P1', 'P0']])

# Count how many P1 values are above 0.5
above_05 = (test_data['P1'] > 0.5).sum()

# Count how many P1 values are below or equal to 0.5
below_05 = (test_data['P1'] <= 0.5).sum()

# Print results
print(f"Number of cases where P1 > 0.5: {above_05}")
print(f"Number of cases where P1 ≤ 0.5: {below_05}")

# Convert probabilities into predicted choices
test_data['predicted_choice'] = (test_data['P1'] > 0.5).astype(int)  # 1 if P1 > 0.5, else 0

# Compare with actual choices
correct_predictions = (test_data['predicted_choice'] == test_data['binary_seats']).sum()

# Compute accuracy
accuracy = correct_predictions / len(test_data)

# Print results
print(f"Correct Predictions: {correct_predictions} / {len(test_data)}")
print(f"Model Accuracy: {accuracy:.2%}")

In [ ]:
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
import biogeme.database as db
import pandas as pd

# Charger les données (remplace par ton fichier)
df = pd.read_csv("tes_donnees.csv")  
database = db.Database("my_data", df)

# Définition des variables explicatives
X1 = Variable("X1")
X2 = Variable("X2")
X3 = Variable("X3")

CHOICE = Variable("CHOICE")  # Variable indiquant le choix observé (1,2,3,4 ou 5)
GROUP = Variable("GROUP")  # Variable indiquant l’appartenance au groupe (1 ou 2)

# Définition des paramètres à estimer
ASC_1 = Beta("ASC_1", 0, None, None, 0)
ASC_2 = Beta("ASC_2", 0, None, None, 0)
ASC_3 = Beta("ASC_3", 0, None, None, 0)
ASC_4 = Beta("ASC_4", 0, None, None, 0)
ASC_5 = Beta("ASC_5", 0, None, None, 0)

B_X1 = Beta("B_X1", 0, None, None, 0)
B_X2 = Beta("B_X2", 0, None, None, 0)
B_X3 = Beta("B_X3", 0, None, None, 0)

# Définition des utilités
V1 = ASC_1 + B_X1 * X1 + B_X2 * X2 + B_X3 * X3
V2 = ASC_2 + B_X1 * X1 + B_X2 * X2 + B_X3 * X3
V3 = ASC_3 + B_X1 * X1 + B_X2 * X2 + B_X3 * X3
V4 = ASC_4 + B_X1 * X1 + B_X2 * X2 + B_X3 * X3
V5 = ASC_5 + B_X1 * X1 + B_X2 * X2 + B_X3 * X3

# Dictionnaire des utilités
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5}

# Définition des conditions de disponibilité
av = {
    1: (GROUP == 1),  # Alternatives 1, 2, 3 sont disponibles pour le groupe 1
    2: (GROUP == 1),
    3: (GROUP == 1),
    4: (GROUP == 2),  # Alternatives 4, 5 sont disponibles pour le groupe 2
    5: (GROUP == 2),
}

# Définition du modèle logit conditionnel
logprob = bio.bioLogLogit(V, av, CHOICE)

# Création de l’objet Biogeme
biogeme = bio.Biogeme(database, logprob)
biogeme.modelName = "MNL_Deux_Groupes"

# Estimation
results = biogeme.estimate()

# Affichage des résultats
print(results.getEstimatedParameters())

In [ ]:
import pandas as pd
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable

# Load the dataset
df = pd.read_csv("your_data.csv")  # Replace with your actual dataset file

# Create a unique ID for each choice situation (group)
df["Group_ID"] = df.groupby(["departure", "arrival", "departure_date_num"]).ngroup()

# Define variables
price = Variable('Price')
bus_distance = Variable('Bus_Distance')
safety = Variable('Safety')
binary_seats = Variable('binary_seats')

# Define model parameters (to be estimated)
beta_price = Beta('beta_price', 0, None, None, 0)
beta_distance = Beta('beta_distance', 0, None, None, 0)
beta_safety = Beta('beta_safety', 0, None, None, 0)

# Define the number of alternatives (replace with your actual number)
num_alternatives = 500  

# Define utility functions and availability conditions
V = {}
av = {}

for i in range(1, num_alternatives + 1):  
    V[i] = beta_price * Variable(f'Price_{i}') + \
           beta_distance * Variable(f'Bus_Distance_{i}') + \
           beta_safety * Variable(f'Safety_{i}')
    
    av[i] = Variable(f'Availability_{i}')  # Binary column indicating availability

# Define the choice model (logit with availability constraints)
logit_model = models.logit(V, av, binary_seats)

# Create Biogeme object with groups
biogeme = bio.Biogeme(df, logit_model)
biogeme.modelName = "mnl_model"
biogeme.panelColumn = "Group_ID"  # Tells Biogeme to treat data as grouped by Group_ID

# Estimate parameters
results = biogeme.estimate()

# Display results
print(results.getEstimatedParameters())
